# Transcribe any audio stream with Deepgram!

**Make a copy of this notebook into your own drive, and follow the instructions below!** 🥳🥳🥳


----------------------------

# Get started:

1) Copy this notebook (`File > Save a copy in Drive`) or download the .ipynb (`File > Download > Download as .ipynb`). 

2) Follow the instructions below!

----------------------------
# Instructions:
Running the following cells will allow you to transcribe any audio stream you wish. The comments below point out the variables you can manipulate to modify your output.

Also, this notebook comes with a video tutorial! Here: https://youtu.be/dq4AiiiaAsY

And by the way, if you haven't yet signed up for Deepgram, check out this link: https://dpgr.am/streaming-notebook

# Step 1: Dependencies

Run this cell to download all necessary dependencies.

Note: You can run a cell by clicking the play button on the left or by clicking on the cell and pressing `shift`+`ENTER` at the same time. (Or `shift` + `return` on Mac).

In [ ]:
! pip install deepgram-sdk --upgrade

# Step 2: Live Transcription

Run this cell to livestream audio from any source! By default, we are streaming from BBC radio:  'http://stream.live.vc.bbcmedia.co.uk/bbc_radio_fourlw_online_nonuk'

Set the following variables as necessary:


*   `DEEPGRAM_API_KEY` = Your personal Deepgram API key
*   `URL` = The URL you wish to stream from. Again, we're streaming from BBC Radio by default, but if you wish to stream from another source just replace this URL.
*  `PARAMS` = The starter code here should give you a well-punctuated English transcript. However, if you wish to change the language, use a different model, or modify the output, check out our docs to figure out the best parameter configuration for you: https://developers.deepgram.com/documentation/
*  `TRANSCRIPTION_DURATION` = The number of seconds you wish to transcribe the livestream. Set this equal to `float(inf)` if you wish to stream forever. (Or at least until you wish to cut off the function manually.)
*  `TRANSCRIPT_ONLY` = Set this variable to `True` if you wish only to see the words you're transcribing. Set it to `False` if you wish to see the full JSON responses (which include word-level timestamps, metadata, and confidence levels).

To run the cell, click the play button or press `Shift`+`Enter` on your keyboard. (Or `Shift`+`return` if you're on Mac.)

-------

Note: if you're already a Deepgram user, and you're getting an error in this cell the most common issues are:

1. You may need to update your installation of `deepgram-sdk`.
2. You may need to check how many credits you have left in your Deepgram account.



In [ ]:
import httpx
import threading
import time
import json

from deepgram import (
    DeepgramClient,
    LiveTranscriptionEvents,
    LiveOptions,
)

# Your Deepgram API key
DEEPGRAM_API_KEY = '🔑🔑🔑 Your API Key here! 🔑🔑🔑'

# URL for the realtime streaming audio you would like to transcribe
URL = 'http://stream.live.vc.bbcmedia.co.uk/bbc_world_service'

# Duration for transcription in seconds
TRANSCRIPTION_DURATION = 30

# Set this variable to `True` if you wish only to 
# see the transcribed words, like closed captions. 
# Set it to `False` if you wish to see the raw JSON responses.
TRANSCRIPT_ONLY = False

def main():
    try:
        # Initialize Deepgram client
        deepgram = DeepgramClient(DEEPGRAM_API_KEY)

        # Create a websocket connection to Deepgram
        dg_connection = deepgram.listen.live.v("1")

        # Define event handlers
        def on_message(self, result, **kwargs):
            if TRANSCRIPT_ONLY:
                sentence = result.channel.alternatives[0].transcript
                if len(sentence) > 0:
                    print(f"Speaker: {sentence}")
            else:
                # Extract relevant information from the LiveResultResponse object
                response_dict = {
                    "type": result.type,
                    "channel": {
                        "alternatives": [
                            {
                                "transcript": alt.transcript
                            } for alt in result.channel.alternatives
                        ]
                    }
                }
                print(json.dumps(response_dict, indent=4))


        def on_metadata(self, metadata, **kwargs):
            print(f"\n\n{metadata}\n\n")

        def on_error(self, error, **kwargs):
            print(f"\n\n{error}\n\n")

        # Register event handlers
        dg_connection.on(LiveTranscriptionEvents.Transcript, on_message)
        dg_connection.on(LiveTranscriptionEvents.Metadata, on_metadata)
        dg_connection.on(LiveTranscriptionEvents.Error, on_error)

        # Configure Deepgram options for live transcription
        options = LiveOptions(
            model="nova-2", 
            language="en-US", 
            smart_format=True,
        )

        # Start the connection
        dg_connection.start(options)

        # Create a lock and a flag for thread synchronization
        lock_exit = threading.Lock()
        exit = False

        # Define a thread that streams the audio and sends it to Deepgram
        def myThread():
            start_time = time.time()
            with httpx.stream("GET", URL) as r:
                for data in r.iter_bytes():
                    if time.time() - start_time >= TRANSCRIPTION_DURATION:
                        break
                    dg_connection.send(data)

        # Start the thread
        myHttp = threading.Thread(target=myThread)
        myHttp.start()

        # Wait for the specified duration
        myHttp.join(TRANSCRIPTION_DURATION)

        # Set the exit flag to True to stop the thread
        lock_exit.acquire()
        exit = True
        lock_exit.release()

        # Close the connection to Deepgram
        dg_connection.finish()

        print("Finished")

    except Exception as e:
        print(f"Could not open socket: {e}")
        return


if __name__ == "__main__":
    main()
